In [1]:
import pandas as pd
import numpy as np

In [2]:
# load dataset
from sklearn.datasets import load_breast_cancer

data = load_breast_cancer()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = pd.Series(data.target, name='target')

In [3]:
X.head()

,mean radius,mean texture,mean perimeter,mean area,mean smoothness,mean compactness,mean concavity,mean concave points,mean symmetry,mean fractal dimension,...,worst radius,worst texture,worst perimeter,worst area,worst smoothness,worst compactness,worst concavity,worst concave points,worst symmetry,worst fractal dimension
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678


In [4]:
# train-test split
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
#scaling
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [6]:
# label encoding - target
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train = le.fit_transform(y_train)
y_test = le.transform(y_test)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim

In [32]:
# converting numpy array to torch tensors

X_train_tensor = torch.tensor(X_train)
print("X Train Tensor Shape", X_train_tensor.shape)
X_test_tensor = torch.tensor(X_test)
print("X Test Tensor Shape", X_test_tensor.shape)
y_train_tensor = torch.tensor(y_train)
print("Y Train Tensor Shape", y_train_tensor.shape)
y_test_tensor = torch.tensor(y_test)
print("Y Test Tensor Shape", y_test_tensor.shape)

X Train Tensor Shape torch.Size([455, 30])
X Test Tensor Shape torch.Size([114, 30])
Y Train Tensor Shape torch.Size([455])
Y Test Tensor Shape torch.Size([114])


In [28]:
class SimpleNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__() # callinf the parent's constructor
        self.network = nn.Sequential(
            nn.Linear(input_size, hidden_size),
            nn.ReLU(),
            nn.Linear(hidden_size, output_size),
            nn.Sigmoid()
        )
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_epochs = 25
        
    def forward(self, x):
        out = self.network(x)
        return out
    
    def loss_function(self, y_pred, y):
        criterion = nn.BCELoss() # binary cross entropy loss
        return criterion(y_pred, y.float())

    def optimizer_function(self, learning_rate=0.001):
        return optim.Adam(self.parameters(), lr=learning_rate)
    
    def train_nn(self, X, y, learning_rate=0.001):
        optimizer = self.optimizer_function(learning_rate)
        for epoch in range(self.num_epochs):
            # forward pass
            y_pred = self.forward(X)
            loss = self.loss_function(y_pred, y)
            # backward pass and optimization
            # removing previous gradient
            optimizer.zero_grad() 
            # calculating gardienta
            loss.backward() 
            # update the weights
            optimizer.step() 

            print(f"Epoch [{epoch+1}/{self.num_epochs}], Loss: {loss.item():.4f}")

In [29]:
model = SimpleNN(input_size=X_train.shape[1], hidden_size=16, output_size=1)

In [33]:
y_train_tensor = y_train_tensor.float().unsqueeze(1)

In [34]:
model.train_nn(X_train_tensor.float(), y_train_tensor, learning_rate=0.001)

Epoch [1/25], Loss: 0.5294
Epoch [2/25], Loss: 0.5229
Epoch [3/25], Loss: 0.5164
Epoch [4/25], Loss: 0.5100
Epoch [5/25], Loss: 0.5036
Epoch [6/25], Loss: 0.4973
Epoch [7/25], Loss: 0.4911
Epoch [8/25], Loss: 0.4849
Epoch [9/25], Loss: 0.4787
Epoch [10/25], Loss: 0.4727
Epoch [11/25], Loss: 0.4667
Epoch [12/25], Loss: 0.4607
Epoch [13/25], Loss: 0.4548
Epoch [14/25], Loss: 0.4489
Epoch [15/25], Loss: 0.4431
Epoch [16/25], Loss: 0.4374
Epoch [17/25], Loss: 0.4317
Epoch [18/25], Loss: 0.4260
Epoch [19/25], Loss: 0.4204
Epoch [20/25], Loss: 0.4148
Epoch [21/25], Loss: 0.4093
Epoch [22/25], Loss: 0.4039
Epoch [23/25], Loss: 0.3985
Epoch [24/25], Loss: 0.3931
Epoch [25/25], Loss: 0.3879


In [35]:
y_test_pred_labels = model.forward(X_test_tensor.float())
y_test_pred_labels = (y_test_pred_labels >= 0.5).float()

In [36]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

accuracy = accuracy_score(y_test_tensor, y_test_pred_labels)
precision = precision_score(y_test_tensor, y_test_pred_labels)
recall = recall_score(y_test_tensor, y_test_pred_labels)
f1 = f1_score(y_test_tensor, y_test_pred_labels)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")

Accuracy: 0.9561
Precision: 0.9342
Recall: 1.0000
F1 Score: 0.9660
